In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from keras import backend as K
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
import os
import time

input_data = pd.read_csv("TRAINING_50K_input.csv")
output_data = pd.read_csv("TRAINING_50K_output.csv")
#time = np.linspace(0.0001, 0.5, num=5000)
test = pd.read_csv("paper-test.csv").to_numpy()
X_test = test[:-1]
Y_test = test[1:]
feature_cols = ["T", "Y_h", "Y_h2", "Y_o", "Y_o2", "Y_oh", "Y_h2o", "Y_ho2", "Y_h2o2"]
X = input_data[feature_cols].to_numpy()
X_next = output_data[feature_cols].to_numpy()

# Check for NaN values
assert not np.any(np.isnan(X)), "X contains NaN values"
assert not np.any(np.isnan(X_next)), "y contains NaN values"

X, X_next = shuffle(X, X_next)

t_test = np.linspace(0, 2500, 1)

print(X.shape)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-11-01 10:04:24.211469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(47405, 9)


In [2]:
class BaseModel(tf.keras.Model):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(16, activation='relu')
        self.dense2 = tf.keras.layers.Dense(32, activation='relu')
        self.dense3 = tf.keras.layers.Dense(16, activation='relu')
        self.output_layer = tf.keras.layers.Dense(9)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return self.output_layer(x)


In [3]:
class DeepEnsemble:
    def __init__(self, n_models=10, model_dir='saved_models'):
        self.n_models = n_models
        self.models = [BaseModel() for _ in range(n_models)]
        self.model_dir = model_dir
        self.state_dim = 9  # Define state dimension

    def compile(self, optimizer='adam', loss='mse'):
        for model in self.models:
            model.compile(optimizer=optimizer, loss=loss)

    def fit(self, dataset, epochs=500, verbose=0, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)]):
        for i, model in enumerate(self.models):
            print(f'Training model {i+1}/{self.n_models}')
            model.fit(dataset, epochs=epochs, verbose=verbose, callbacks=callbacks)

    def predict(self, X):
        predictions = np.array([model.predict(X, verbose=0) for model in self.models])
        mean = np.mean(predictions, axis=0)
        variance = np.var(predictions, axis=0)
        return mean, variance
    
    def predict_iterative(self, initial_state, n_steps, include_variance=True):
        """
        Perform iterative predictions starting from an initial state.
        
        Args:
            initial_state: numpy array of shape (9,) containing the state variables
            n_steps: number of time steps to predict
            include_variance: whether to return variance estimates
            
        Returns:
            predictions: numpy array of shape (n_steps, 9) containing state predictions
            variances: numpy array of shape (n_steps, 9) containing variance estimates
                      (only if include_variance=True)
        """
        # Verify input dimension
        if initial_state.shape != (self.state_dim,):
            raise ValueError(f"Initial state must have shape ({self.state_dim},), got {initial_state.shape}")
        
        # Initialize arrays to store predictions and variances
        predictions = np.zeros((n_steps, self.state_dim))
        if include_variance:
            variances = np.zeros((n_steps, self.state_dim))
        
        # Set initial state
        current_state = initial_state.reshape(1, -1)  # Reshape to (1, 9) for model input
        
        # Perform iterative predictions
        for step in range(n_steps):
            # Get prediction and variance for current state
            mean, variance = self.predict(current_state)
            
            # Store results
            predictions[step] = mean[0]  # Remove batch dimension
            if include_variance:
                variances[step] = variance[0]
            
            # Update current state for next prediction
            current_state = mean
        
        if include_variance:
            return predictions, variances
        return predictions
    
    def save_models(self, custom_dir=None):
        """Save all models in the ensemble"""
        save_dir = custom_dir if custom_dir else self.model_dir
        os.makedirs(save_dir, exist_ok=True)
        
        for i, model in enumerate(self.models):
            model_path = os.path.join(save_dir, f'model_{i}')
            model.save(model_path)
        
        metadata = {
            'n_models': self.n_models,
            'model_dir': save_dir,
            'state_dim': self.state_dim
        }
        np.save(os.path.join(save_dir, 'metadata.npy'), metadata)
        
        print(f'Ensemble saved to {save_dir}')

    @classmethod
    def load_models(cls, model_dir):
        """Load a previously saved ensemble"""
        if not os.path.exists(model_dir):
            raise ValueError(f"Directory {model_dir} does not exist")
            
        metadata = np.load(os.path.join(model_dir, 'metadata.npy'), allow_pickle=True).item()
        ensemble = cls(n_models=metadata['n_models'], model_dir=model_dir)
        
        ensemble.models = []
        for i in range(metadata['n_models']):
            model_path = os.path.join(model_dir, f'model_{i}')
            model = tf.keras.models.load_model(model_path)
            ensemble.models.append(model)
            
        print(f'Loaded ensemble from {model_dir}')
        return ensemble

In [4]:
def plot_results(t_test, x_test, mean, variance):
    plt.figure(figsize=(12, 6))
    plt.plot(t_test, x_test[:, 0], 'b-', label='True x')
    plt.plot(t_test, mean[:, 0], 'r--', label='Predicted x')
    plt.fill_between(t_test, mean[:, 0] - 2*np.sqrt(variance[:, 0]), 
                     mean[:, 0] + 2*np.sqrt(variance[:, 0]), color='r', alpha=0.2)
    plt.xlabel('Time') 
    plt.ylabel('Temperature (K)')
    plt.legend()
    plt.show()

def plot_iterative_results(predictions, variances=None, dt=0.1):
    """
    Plot the results of iterative predictions for 9D state.
    
    Args:
        predictions: numpy array of shape (n_steps, 9) containing state predictions
        variances: numpy array of shape (n_steps, 9) containing variance estimates
        dt: time step size for creating time array
    """
    # Create time array
    t = np.arange(len(predictions)) * dt
    
    # Calculate number of rows needed for 9 subplots (3x3 grid)
    n_rows = 3
    n_cols = 3
    
    # Create figure with subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 15))
    
    # Flatten axes for easier iteration
    axes_flat = axes.flatten()
    
    # Plot each state variable
    for i in range(9):
        ax = axes_flat[i]
        
        # Plot predictions
        ax.plot(t, predictions[:, i], 'b-', label=f'State {i+1}')
        
        # Add uncertainty bounds if available
        if variances is not None:
            ax.fill_between(t,
                          predictions[:, i] - 2*np.sqrt(variances[:, i]),
                          predictions[:, i] + 2*np.sqrt(variances[:, i]),
                          color='b', alpha=0.2)
        
        # Customize subplot
        ax.set_xlabel('Time Step')
        ax.set_ylabel(f'State {i+1}')
        ax.grid(True)
        ax.legend()
    
    plt.tight_layout()
    plt.show()

def plot_state_correlation(predictions, state_indices, dt=0.1):
    """
    Plot correlation between two selected state variables.
    
    Args:
        predictions: numpy array of shape (n_steps, 9) containing state predictions
        state_indices: tuple of two integers specifying which states to plot
        dt: time step size for creating time array
    """
    i, j = state_indices
    
    plt.figure(figsize=(8, 8))
    plt.plot(predictions[:, i], predictions[:, j], 'b-')
    plt.xlabel(f'State {i+1}')
    plt.ylabel(f'State {j+1}')
    plt.title(f'State {i+1} vs State {j+1}')
    plt.grid(True)
    plt.axis('equal')
    plt.show()

In [5]:
def plot_iterative_results(predictions, variances=None, dt=0.1):
    """
    Plot the results of iterative predictions for 9D state.
    
    Args:
        predictions: numpy array of shape (n_steps, 9) containing state predictions
        variances: numpy array of shape (n_steps, 9) containing variance estimates
        dt: time step size for creating time array
    """
    # Create time array
    t = np.arange(len(predictions)) * dt
    
    # Calculate number of rows needed for 9 subplots (3x3 grid)
    n_rows = 3
    n_cols = 3
    
    # Create figure with subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 15))
    
    # Flatten axes for easier iteration
    axes_flat = axes.flatten()
    
    # Plot each state variable
    for i in range(9):
        ax = axes_flat[i]
        
        # Plot predictions
        ax.plot(t, predictions[:, i], 'b-', label=f'State {i+1}')
        
        # Add uncertainty bounds if available
        if variances is not None:
            ax.fill_between(t,
                          predictions[:, i] - 2*np.sqrt(variances[:, i]),
                          predictions[:, i] + 2*np.sqrt(variances[:, i]),
                          color='b', alpha=0.2)
        
        # Customize subplot
        ax.set_xlabel('Time Step')
        ax.set_ylabel(f'State {i+1}')
        ax.grid(True)
        ax.legend()
    
    plt.tight_layout()
    plt.show()

def plot_state_correlation(predictions, state_indices, dt=0.1):
    """
    Plot correlation between two selected state variables.
    
    Args:
        predictions: numpy array of shape (n_steps, 9) containing state predictions
        state_indices: tuple of two integers specifying which states to plot
        dt: time step size for creating time array
    """
    i, j = state_indices
    
    plt.figure(figsize=(8, 8))
    plt.plot(predictions[:, i], predictions[:, j], 'b-')
    plt.xlabel(f'State {i+1}')
    plt.ylabel(f'State {j+1}')
    plt.title(f'State {i+1} vs State {j+1}')
    plt.grid(True)
    plt.axis('equal')
    plt.show()

In [6]:
def train_model():
    dataset = tf.data.Dataset.from_tensor_slices((X, X_next)).shuffle(47405).batch(32)
    ensemble = DeepEnsemble(n_models=10)
    ensemble.compile()
    ensemble.fit(dataset)

    return ensemble

In [7]:
def evaluate_model(model, X_test, Y_test):
    mean, variance = model.predict(X_test)
    mse = np.mean((Y_test - mean)**2)
    uncertainty = np.mean(np.sqrt(variance))
    return mse, uncertainty

In [ ]:
def main():
    
    #X, X_next, X_test, Y_test, t_test = import_data()
    # Define paths
    saved_model_path = 'my_saved_ensemble'

    # Check for existing model
    if not os.path.exists(saved_model_path):
        print("No existing model found. Training new ensemble...")
        try:
            # Train the model using train_model()
            ensemble = train_model()
            
            # Save the trained ensemble
            print("Training complete. Saving model...")
            ensemble.save_models(saved_model_path)
            print(f"Model saved to {saved_model_path}")
            
        except Exception as e:
            print(f"Error during training: {str(e)}")
            return
    else:
        # Load the trained model
        print("Loading existing ensemble...")
        try:
            ensemble = DeepEnsemble.load_models(saved_model_path)
            print("Ensemble loaded successfully")
        except Exception as e:
            print(f"Error loading model: {str(e)}")
            return

    # Define initial condition (9-dimensional state)
    initial_state = X[0, :]
    
    # Perform iterative predictions
    n_steps = 250
    print(f"Performing iterative predictions for {n_steps} steps...")
    try:
        predictions, variances = ensemble.predict_iterative(initial_state, n_steps=n_steps)
        
        # Visualize the results
        print("Plotting results...")
        plot_iterative_results(predictions, variances, dt=0.1)
        plot_state_correlation(predictions, (0, 1))
        
    except Exception as e:
        print(f"Error during prediction or plotting: {str(e)}")

if __name__ == "__main__":
    main()

No existing model found. Training new ensemble...
Training model 1/10


2024-11-01 10:04:28.648135: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Training model 2/10


2024-11-01 10:15:33.669815: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 43791 of 47405
2024-11-01 10:15:36.162335: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.
2024-11-01 10:22:07.909595: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 43402 of 47405
2024-11-01 10:22:08.147873: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.
2024-11-01 10:22:33.640888: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 43616 of 47405
2024-11-01 10:22:34.713655: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.
2024-11-01 10:24:33.583006: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 45142 of 47405
2024-11-01 10:24:36.557668: I tensorflow/core/kernels/data/shuffl

Training model 3/10
Training model 4/10
Training model 5/10


In [ ]:
t_test = np.linspace(0, 1, 2499)
test = pd.read_csv("paper-test.csv").to_numpy()
X_test = test[:-1]
Y_test = test[1:]
def plot_results(t_test, x_test, mean, variance):
    plt.figure(figsize=(12, 6))
    plt.plot(t_test.T, x_test[:, 0], 'b-', label='True x')
    plt.plot(t_test.T, mean[:, 0], 'r--', label='Predicted x')
    plt.fill_between(t_test, mean[:, 0] - 2*np.sqrt(variance[:, 0]), 
                     mean[:, 0] + 2*np.sqrt(variance[:, 0]), color='r', alpha=0.2)
    plt.xlabel('Time') 
    plt.ylabel('Temperature (K)')
    plt.legend()
    plt.show()

In [ ]:
plot_results(t_test, X_test, mean, variance)

In [ ]:
plt.plot(t_test.T, X_test[:, 8], 'b-', label='True x')